## AutoEncoders on PointClouds - Modular Code

In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet

from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.point_net_ae import Configuration as PN_Conf

from general_tools.in_out.basics import create_dir, delete_files_in_directory
from geo_tool import Point_Cloud

Loading manual conv_3d_transpose.
nn_distance module cannot be loaded.


In [15]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

data_path = '/Users/optas/DATA/Point_Clouds/Shape_Net_Core/from_manifold_meshes/1024/03001627'
train_dir = '/Users/optas/DATA/Neural_Nets/Models/Point_Cloud_AE/'
experiment_name = 'scratch'

train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)
# delete_files_in_directory(train_dir)

In [4]:
file_names = pio.load_filenames_of_input_data(data_path)
file_names = file_names[:1000]

pclouds, model_names = pio.load_crude_point_clouds(file_names=file_names, n_threads=11)




train_data_, val_data_, test_data_ = pio.train_validate_test_split([pclouds, model_names],
                                                                   train_perc=0.8,
                                                                   validate_perc=0.1,
                                                                   test_perc=0.1,
                                                                   seed=seed)

train_data = PointCloudDataSet(train_data_[0], labels=train_data_[1])
val_data = PointCloudDataSet(val_data_[0], labels=val_data_[1])
test_data = PointCloudDataSet(test_data_[0], labels=test_data_[1])

6778 files containing  point clouds were found.


In [5]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

conf = PN_Conf(n_input = [1024, 3],
               training_epochs = 16,
               batch_size = 20,
               loss = 'l2',
               train_dir = train_dir,
               loss_display_step = 1,               
               saver_step = 2,
               learning_rate = 0.0002,
               debug = True
          )

In [7]:
reset_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

if conf.debug:
    ae.train_names = set(train_data_[1])

ae.train(train_data, conf)

In [20]:
reset_graph()
ae = PointNetAutoEncoder(experiment_name, conf)
ae.restore_model(conf.train_dir, 10)

In [19]:
# Verifying that by mixing the test/val the same model will prefer a different set, each time.
from tf_lab.point_clouds.in_out import shuffle_two_pcloud_datasets
new_test_data, new_val_data = shuffle_two_pcloud_datasets(test_data, val_data)
_, loss_tes, _ = ae.evaluate(new_test_data, conf)
_, loss_val, _ = ae.evaluate(new_val_data, conf)
if loss_val < loss_tes:
    print 'val winner'
else:
    print 'test winner'

val winner


In [37]:
# Experimenting with Tensorboard - Visualizations
from tf_lab.fundamentals.inspect import hist_summary_of_trainable
writer = tf.summary.FileWriter('/Users/optas/Desktop/test', ae.graph)
# hist_summary_of_trainable(ae.graph)